# 🤖 Análise de Consenso entre LLMs - Anotação sem LLM Hacking

## 1) Setup e Configuração

In [ ]:
# Imports
import sys
from loguru import logger
import pandas as pd

# Configurar logging
logger.remove()
logger.add(
    sys.stdout,
    format="<green>{time:HH:mm:ss}</green> | <level>{level: <8}</level> | <level>{message}</level>",
    level="INFO"
)

logger.success("✓ Setup completo")

18:23:09 | SUCCESS  | ✓ Setup completo


## 2) Carregar Dataset

In [2]:
from src.utils.data_loader import load_hf_dataset, load_hf_dataset_as_dataframe, list_available_datasets, get_dataset_info

# Listar datasets
logger.info("Datasets disponíveis:")
for dataset in list_available_datasets():
    logger.info(f"  - {dataset}")

18:23:09 | Datasets disponíveis:
18:23:09 |   - agnews
18:23:09 |   - mpqa
18:23:09 |   - webkb
18:23:09 |   - ohsumed
18:23:09 |   - acm
18:23:09 |   - yelp_2013
18:23:09 |   - dblp
18:23:09 |   - books
18:23:09 |   - reut90
18:23:09 |   - wos11967
18:23:09 |   - twitter
18:23:09 |   - trec
18:23:09 |   - wos5736
18:23:09 |   - sst1
18:23:09 |   - pang_movie
18:23:09 |   - movie_review
18:23:09 |   - vader_movie
18:23:09 |   - subj
18:23:09 |   - sst2
18:23:09 |   - yelp_reviews
18:23:09 |   - 20ng
18:23:09 |   - medline


In [3]:
dataset_name = "yelp_reviews"  # Ajuste conforme necessário

texts, categories, ground_truth = load_hf_dataset(dataset_name)

logger.info(f"Textos: {len(texts)}")
logger.info(f"Categorias: {categories}")
logger.info(f"Ground truth: {'Sim' if ground_truth else 'Não'}")

18:23:09 | Carregando dataset: yelp_reviews
18:23:17 | Split 'train': 45000 exemplos
18:23:17 | Categorias extraídas automaticamente: [0, 1]
18:23:17 | Amostra reduzida para 100 exemplos
18:23:17 | Coluna de texto: text
18:23:17 | Ground truth carregado da coluna 'label'
18:23:17 | Textos: 100
18:23:17 | Categorias: [0, 1]
18:23:17 | Ground truth: Sim


In [4]:
# Visualizar amostra
logger.info("Amostra dos textos:")
for i, text in enumerate(texts[:3]):
    logger.info(f"{i+1}. {text[:100]}...")
    if ground_truth:
        logger.info(f"   Label: {ground_truth[i]}")

18:23:17 | Amostra dos textos:
18:23:17 | 1. I love Marilo  She understands my hair type and knows exactly what to do with my hair  She keeps a r...
18:23:17 |    Label: 1
18:23:17 | 2. I went to this Petco a few times and was a bit appalled at their rodents section to be perfectly hon...
18:23:17 |    Label: 0
18:23:17 | 3. I love coming to this bar Friendly bartenders and a beautiful view.
...
18:23:17 |    Label: 1


In [5]:
df, categories = load_hf_dataset_as_dataframe(dataset_name)

18:23:17 | Carregando dataset: yelp_reviews
18:23:23 | Split 'train': 45000 exemplos
18:23:23 | Categorias extraídas automaticamente: [0, 1]
18:23:23 | Amostra reduzida para 100 exemplos
18:23:23 | Coluna de texto: text
18:23:23 | Ground truth carregado da coluna 'label'
18:23:23 | DataFrame criado com 100 linhas


In [6]:
categories

[0, 1]

## 3) Configurar Modelos LLM

### Opção A: Usar apenas parâmetros padrão (temp=0)

In [7]:
from src.llm_annotation_system.annotation.llm_annotator import LLMAnnotator
from src.experiments.base_experiment import DEFAULT_MODELS, BASE_ANNOTATION_PROMPT

annotator = LLMAnnotator(
    dataset_name=dataset_name,
    categories=categories,
    models=DEFAULT_MODELS,
    prompt_template=BASE_ANNOTATION_PROMPT,
    use_langchain_cache=True,
    use_alternative_params=False  # Apenas temp=0
)

logger.success(f"✓ Annotator inicializado com {len(annotator.models)} modelos")

18:23:24 | Cache carregado: 220 entradas
18:23:24 | Cache LangChain ativado: ..\..\data\.cache\langchain\langchain_cache.db
18:23:24 | Template do prompt preparado
18:23:24 | LLMAnnotator inicializado
18:23:24 | Modelos: 3 | Categorias: 2
18:23:24 | ✓ Annotator inicializado com 3 modelos


## 4) Executar Anotação

In [8]:
df

,text,label,label_description
0,I love Marilo She understands my hair type an...,1,Positive Sentiment
1,I went to this Petco a few times and was a bit...,0,Negative Sentiment
2,I love coming to this bar Friendly bartenders ...,1,Positive Sentiment
3,Great sushi rolls and BYOB Very friendly staf...,1,Positive Sentiment
4,The sushi is amazing It is made daily in the ...,1,Positive Sentiment
...,...,...,...
95,Bombebero's you've won my five star rating I a...,1,Positive Sentiment
96,Great food coffee deserts Also pet friendly o...,1,Positive Sentiment
97,I love the food this place serves I had the Ba...,1,Positive Sentiment
98,Nice local neighborhood place to come and enjo...,1,Positive Sentiment


### - Testar anotação única

In [9]:
# Parâmetros de anotação
num_repetitions = 3
text = texts[0]
model = "llama3.1-8b"

total_annotation = len(text) * num_repetitions
logger.warning(f"  Modelo: {model}")
logger.warning(f"  Texto: {len(text)}")
logger.warning(f"  Repetições: {num_repetitions}")

annotations = annotator.annotate_single(
    text=text,
    model=model,
    num_repetitions=num_repetitions,
    use_cache=False
)

logger.success("✓ Anotação completa")

annotations

18:23:24 |   Modelo: llama3.1-8b
18:23:24 |   Texto: 374
18:23:24 |   Repetições: 3
18:23:25 | ✓ Anotação completa


[1, 1, 1]

### - Anotando dataset completo

In [10]:
output_file = annotator.results_dir / "annotations_complete.csv"
use_old_annotations = False

if use_old_annotations and output_file.exists():
    logger.warning(f"Arquivo já existe: {output_file}")
    logger.warning("Carregando anotações salvas e pulando anotação...")
    df_annotations = pd.read_csv(output_file)
    logger.success("✓ Anotações carregadas do disco")
    
else: 
    num_repetitions = 1
    df_annotations = annotator.annotate_dataset(
        texts=texts,
        num_repetitions=num_repetitions,
        use_cache=False
    )

logger.success("✓ Anotações completas")
df_annotations

18:23:25 | Iniciando anotação
18:23:25 | Textos: 100 | Modelos: 3 | Repetições: 1
18:23:25 | Total de anotações: 300
Anotando: 100%|██████████| 100/100 [00:31<00:00,  3.22it/s]
18:23:56 | Anotações completas salvas: ..\..\results\yelp_reviews\annotations_complete.csv
18:23:56 | ✓ Anotações completas


,text_id,text,gemma3-4b_rep1,gemma3-4b_consensus,gemma3-4b_consensus_score,mistral-7b_rep1,mistral-7b_consensus,mistral-7b_consensus_score,llama3.1-8b_rep1,llama3.1-8b_consensus,llama3.1-8b_consensus_score
0,0,I love Marilo She understands my hair type an...,1,1,1.0,1,1,1.0,1,1,1.0
1,1,I went to this Petco a few times and was a bit...,0,0,1.0,0,0,1.0,0,0,1.0
2,2,I love coming to this bar Friendly bartenders ...,1,1,1.0,1,1,1.0,1,1,1.0
3,3,Great sushi rolls and BYOB Very friendly staf...,1,1,1.0,1,1,1.0,1,1,1.0
4,4,The sushi is amazing It is made daily in the ...,1,1,1.0,1,1,1.0,1,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
95,95,Bombebero's you've won my five star rating I a...,1,1,1.0,1,1,1.0,1,1,1.0
96,96,Great food coffee deserts Also pet friendly o...,1,1,1.0,1,1,1.0,1,1,1.0
97,97,I love the food this place serves I had the Ba...,1,1,1.0,1,1,1.0,1,1,1.0
98,98,Nice local neighborhood place to come and enjo...,1,1,1.0,1,1,1.0,1,1,1.0


### - Metricas por modelo

In [11]:
df_annotations["ground_truth"] = ground_truth if ground_truth else None
df_metrics = annotator.evaluate_model_metrics(
    df_annotations, 
    ground_truth_col="ground_truth", 
    output_csv=True
)

18:23:56 | Calculando métricas por modelo...
18:23:56 | Total de linhas avaliadas: 100
18:23:56 | Métricas para gemma3-4b: Acc=0.9900, F1=0.9902, Prec=0.9907, Rec=0.9900, Cov=1.0000
18:23:56 | Métricas para mistral-7b: Acc=0.9900, F1=0.9902, Prec=0.9907, Rec=0.9900, Cov=1.0000
18:23:56 | Métricas para llama3.1-8b: Acc=0.9800, F1=0.9793, Prec=0.9804, Rec=0.9800, Cov=1.0000
18:23:56 | Métricas por modelo salvas em: ..\..\results\yelp_reviews\model_metrics.csv
18:23:56 | ✓ Métricas calculadas com sucesso


## 5) Calcular Consenso

In [12]:
from src.llm_annotation_system.consensus.consensus_calculator import ConsensusCalculator
from src.llm_annotation_system.consensus.consensus_evaluator import ConsensusEvaluator
from src.experiments.base_experiment import EXPERIMENT_CONFIG


# Inicializar calculador
calculator = ConsensusCalculator(
    consensus_threshold=EXPERIMENT_CONFIG['consensus'].get('threshold', 0.8),
    default_strategy=EXPERIMENT_CONFIG['consensus'].get('strategy', "majority_vote")
)

analyzer = ConsensusEvaluator(categories=categories, calculator=calculator, output_dir=annotator.results_dir)

df_with_consensus = analyzer.compute_consensus(df_annotations)

18:23:56 | Executando cálculo de consenso interno...
18:23:56 | Calculando consenso...
18:23:56 | Consenso calculado:
18:23:56 |   Alto (≥80%): 96 (96.0%)
18:23:56 |   Médio (60-80%): 4 (4.0%)
18:23:56 |   Baixo (<60%): 0 (0.0%)
18:23:56 |   Problemáticos: 0 (0.0%)
18:23:56 |   Itens que precisam de revisão: 0 (0.0%)
18:23:56 | Cálculo de consenso finalizado.


### - Estatisticas

In [13]:
logger.info("\n📊 Estatísticas de Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Mediana: {df_with_consensus['consensus_score'].median():.2%}")
logger.info(f"  Desvio padrão: {df_with_consensus['consensus_score'].std():.2%}")

18:23:56 | 
📊 Estatísticas de Consenso:
18:23:56 |   Média: 98.67%
18:23:56 |   Mediana: 100.00%
18:23:56 |   Desvio padrão: 6.56%


### - Distribuição por nível

In [14]:
# Distribuição por nível
levels = df_with_consensus['consensus_level'].value_counts()
logger.info("Distribuição por nível:")
for level, count in levels.items():
    logger.info(f"  {level}: {count} ({count/len(df_with_consensus):.1%})")

18:23:56 | Distribuição por nível:
18:23:56 |   high: 96 (96.0%)
18:23:56 |   medium: 4 (4.0%)


## - Visualizar consenso

In [15]:
from src.llm_annotation_system.consensus.consensus_visualizer import ConsensusVisualizer

visualizer = ConsensusVisualizer(output_dir=annotator.results_dir)

In [16]:
visualizer.plot_score_and_levels(
    df_with_consensus=df_with_consensus,
    levels=levels
)

✓ Gráfico salvo: score_and_levels.html


## 7️⃣ Análise Detalhada de Consenso

### - Gerando Report

In [17]:
# Gerar relatório
report = analyzer.generate_consensus_report(
    df=df_with_consensus
)

logger.success("✓ Relatório gerado")

18:23:57 | Gerando relatório completo de consenso...
18:23:57 | Fleiss' Kappa: 0.882 (Excelente)
18:23:57 | Relatório de consenso gerado com sucesso.
18:23:57 | ✓ Relatório gerado


### - Pairwise_agreement

In [18]:
logger.info("\n📊 Gerando heatmap de concordância...")
visualizer.plot_agreement_heatmap(
    agreement_df=report['pairwise_agreement'],
    title='Matriz de Concordância entre Modelos',
)

18:23:57 | 
📊 Gerando heatmap de concordância...


✓ Heatmap salvo: agreement_heatmap.html


### - Cohens Kappa

In [19]:
logger.info("\n📊 Gerando heatmap de Cohens_Kappa...")
visualizer.plot_kappa_heatmap(
    kappa_df=report['cohens_kappa']
)

18:23:57 | 
📊 Gerando heatmap de Cohens_Kappa...


✓ Heatmap salvo: kappa_heatmap.html


### - Casos problemáticos

In [20]:
# Casos problemáticos
problematic = report.get('problematic_cases')
if problematic is not None and len(problematic) > 0:
    logger.warning(f"\n⚠️  {len(problematic)} casos problemáticos identificados")
    display(problematic)
else:
    logger.success("\n✓ Nenhum caso problemático identificado")

18:23:57 | 
✓ Nenhum caso problemático identificado


## 8️⃣ Validação com Ground Truth

In [22]:
if ground_truth:
    accuracy, cls_report, cm = analyzer.evaluate_ground_truth(
        df_with_consensus=df_with_consensus
    )

    visualizer.plot_confusion_matrix(
        cm=cm,
        categories=categories
    )
else:
    logger.info("⚠️ Ground truth não disponível – pulando validação")

18:24:08 | 
🎯 Accuracy: 100.00%
18:24:08 | 
Classification Report:


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      1.00      1.00        87

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



✓ Matriz de confusão salva em: ..\..\results\yelp_reviews\graphics\confusion_matrix.html


## 9️⃣ Exportar Resultados

In [23]:
import json
from pathlib import Path

# Criar diretório
results_dir = annotator.results_dir
results_dir = results_dir.joinpath("summary")
results_dir.mkdir(parents=True, exist_ok=True)

# Salvar CSVs
df_with_consensus.to_csv(results_dir / 'dataset_anotado_completo.csv', index=False)
logger.info(f"✓ Salvos: {len(df_with_consensus)} registros")

# Alta confiança
high_conf = df_with_consensus[df_with_consensus['consensus_score'] >= 0.8]
high_conf.to_csv(results_dir / 'alta_confianca.csv', index=False)
logger.info(f"✓ Alta confiança: {len(high_conf)} registros")

# Necessita revisão
low_conf = df_with_consensus[df_with_consensus['consensus_score'] < 0.8]
low_conf.to_csv(results_dir / 'necessita_revisao.csv', index=False)
logger.info(f"✓ Necessita revisão: {len(low_conf)} registros")

# Sumário JSON
summary = {
    'dataset': {
        'name': dataset_name,
        'total_texts': len(texts),
        'categories': categories,
        'has_ground_truth': ground_truth is not None
    },
    'config': {
        'models': DEFAULT_MODELS,
        'total_models': len(annotator.models),
        'use_alternative_params': annotator.use_alternative_params,
        'num_repetitions': num_repetitions,
        'total_annotations': len(texts) * len(annotator.models) * num_repetitions
    },
    'results': {
        'consensus_mean': float(df_with_consensus['consensus_score'].mean()),
        'consensus_median': float(df_with_consensus['consensus_score'].median()),
        'high_consensus': int((df_with_consensus['consensus_level'] == 'high').sum()),
        'medium_consensus': int((df_with_consensus['consensus_level'] == 'medium').sum()),
        'low_consensus': int((df_with_consensus['consensus_level'] == 'low').sum()),
    },
    'metrics': {
        'fleiss_kappa': float(report['fleiss_kappa']),
        'fleiss_interpretation': report['fleiss_interpretation']
    }
}

if ground_truth:
    summary['validation'] = {
        'accuracy': float(accuracy)
    }

with open(results_dir / 'sumario_experimento.json', 'w') as f:
    json.dump(summary, f, indent=2)

logger.success("\n✓ Resultados exportados com sucesso!")

18:42:45 | ✓ Salvos: 100 registros
18:42:45 | ✓ Alta confiança: 96 registros
18:42:45 | ✓ Necessita revisão: 4 registros
18:42:45 | 
✓ Resultados exportados com sucesso!


## 10) Resumo Final

In [ ]:
logger.info("\n" + "="*80)
logger.success("RESUMO DO EXPERIMENTO")
logger.info("="*80)

logger.info(f"\n📊 Dataset: {dataset_name}")
logger.info(f"  Textos: {len(texts)}")
logger.info(f"  Categorias: {len(categories)}")

logger.info(f"\n🤖 Configuração:")
logger.info(f"  Modelos base: {len(DEFAULT_MODELS)}")
logger.info(f"  Total modelos: {len(annotator.models)}")
logger.info(f"  Alternative params: {annotator.use_alternative_params}")
logger.info(f"  Repetições: {num_repetitions}")

logger.info(f"\n📈 Consenso:")
logger.info(f"  Média: {df_with_consensus['consensus_score'].mean():.2%}")
logger.info(f"  Fleiss' Kappa: {report['fleiss_kappa']:.3f} ({report['fleiss_interpretation']})")

if ground_truth:
    logger.info(f"\n🎯 Validação:")
    logger.info(f"  Accuracy: {accuracy:.2%}")

logger.info(f"\n📁 Arquivos gerados em: {results_dir}/")

cache_stats = annotator.get_cache_stats()
logger.info(f"\n💾 Cache: {cache_stats['total_entries']} entradas")

logger.success("\n✅ Análise completa!")

18:20:07 | 
18:20:07 | RESUMO DO EXPERIMENTO
18:20:07 | ================================================================================
18:20:07 | 
📊 Dataset: yelp_reviews
18:20:07 |   Textos: 100
18:20:07 |   Categorias: 2
18:20:07 | 
🤖 Configuração:
18:20:07 |   Modelos base: 3
18:20:07 |   Total modelos: 3
18:20:07 |   Alternative params: False
18:20:07 |   Repetições: 1
18:20:07 | 
📈 Consenso:
18:20:07 |   Média: 98.67%
18:20:07 |   Fleiss' Kappa: 0.882 (Excelente)
18:20:07 | 
🎯 Validação:
18:20:07 |   Accuracy: 100.00%
18:20:07 | 
📁 Arquivos gerados em: results\final/
18:20:07 | 
💾 Cache: 220 entradas
18:20:07 | 
✅ Análise completa!
